In [2]:
import matplotlib.pyplot as plt
import cv2
import os
import sys
import numpy as np
print(sys.version)

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import Image

3.8.5 (default, Jan 27 2021, 15:41:15) 
[GCC 9.3.0]


In [3]:
def getHighest(root):
    numbers = []
    for filename in os.listdir(root):
        if('.txt' in filename):
            #filename.split('pcd')
            n = filename.split('pcd')
            m = n[1].split('cpos.txt')
            c = int(m[0])
            #print(c)
            numbers.append(c)
        else:
            pass
    numbers.sort()
    try:
        print(numbers[-1])
    except:
        return 0
    return numbers[-1]

dataset = 'PATH TO DATASET'
dataset_size = getHighest(dataset)
current_img = ''
save_location = 'ENTER SAVE PATH'

FileNotFoundError: [Errno 2] No such file or directory: 'PATH TO DATASET'

In [4]:
%matplotlib inline  

# Define actions on click
def visualize(rect):
    global current_img
    try:
        img = cv2.imread(current_img)
    except:
        img = current_img
    corner0 = rect[0]
    corner1 = rect[1]
    corner2 = rect[2]
    corner3 = rect[3]
    coord0 = [float(x) for x in corner0.split()]
    coord1 = [float(x) for x in corner1.split()]
    coord2 = [float(x) for x in corner2.split()]
    coord3 = [float(x) for x in corner3.split()]
    #attention BGR in OpenCV
    cv2.line(img, (int(coord0[0]), int(coord0[1])), (int(coord1[0]), int(coord1[1])), (10, 200, 10), 1)
    cv2.line(img, (int(coord1[0]), int(coord1[1])), (int(coord2[0]), int(coord2[1])), (200, 15, 0), 1)
    cv2.line(img, (int(coord2[0]), int(coord2[1])), (int(coord3[0]), int(coord3[1])), (10, 200, 10), 1)
    cv2.line(img, (int(coord3[0]), int(coord3[1])), (int(coord0[0]), int(coord0[1])), (200, 15, 0), 1)
    cv2.circle(img, (int(coord0[0]), int(coord0[1])), 2, (15, 15, 255), 2)#red
    cv2.circle(img, (int(coord1[0]), int(coord1[1])), 2, (15, 255, 0), 2)#green
    cv2.circle(img, (int(coord2[0]), int(coord2[1])), 2, (250, 250, 250), 2)#white
    cv2.circle(img, (int(coord3[0]), int(coord3[1])), 2, (150, 150, 150), 2)#gray
    current_img = img
    update_display(current_img)
    
def update_display(img):
    success, encoded_image = cv2.imencode('.png', img)
    bytes_img = encoded_image.tobytes()
    w.value = bytes_img
    
def on_previous_clicked(b):
    slider.value -= 1
    return

def on_next_clicked(b):
    slider.value += 1
    return

def on_previous_clicked_g(b):
    slider_grasps.value -= 1
    return

def on_next_clicked_g(b):
    slider_grasps.value += 1
    return

def on_vis_clicked(b):
    number_g = slider_grasps.value
    number_g*=4
    corners = upload_grasps(slider.value)
    rect = []
    for j in range(number_g, number_g + 4):
        rect.append(corners[j])
    visualize(rect)
    rect *= 0
    
def upload_grasps(number):
    txt_name = txt_name = 'pcd' + str(number).zfill(4) + 'cpos.txt'
    f = open(os.path.join(dataset, txt_name), "r")
    corners = f.readlines() 
    return corners

def setCurrentImagePath(path):
    global current_img
    current_img = path

def upload_image(number): 
    rgb_name = 'pcd' + str(number).zfill(4) + 'r.png'
    rgb_path = os.path.join(dataset, rgb_name)
    setCurrentImagePath(rgb_path)
    with open(rgb_path, 'rb') as file:
        w.value= file.read()
    corners = upload_grasps(number)
    setSliderMax(slider_grasps, len(corners) / 4)
   
def setSliderMax(slider, max):
    slider.max = max - 1
    
def handle_slider_change(change):
    try:
        upload_image(slider.value)
    except:
        pass #do nothing
    
def handle_slider_grasps_change(change):
    try:
        number_g = slider_grasps.value
        number_g*=4
        corners = upload_grasps(slider.value)
        rect = []
        for j in range(number_g, number_g + 4):
            rect.append(corners[j])
        visualize(rect)
        rect *= 0
    except:
        pass

def on_save_clicked(b):
    global current_img
    global save_location
    try:
        img = cv2.imread(current_img)
    except:
        img = current_img
    name = str(slider.value) + '.png'
    save = os.path.join(save_location, name)
    cv2.imwrite(save, img)
    return

###UI INIT
dump = widgets.Output()
slider = widgets.IntSlider()
slider_grasps = widgets.IntSlider()
slider_grasps.max = 5
slider.max = dataset_size 
previous = widgets.Button(description='previous')
previous_g = widgets.Button(description='previous')
next_g = widgets.Button(description='next')
vis = widgets.Button(description='vis')
next = widgets.Button(description='next')
inference = widgets.Button(description='full', button_style='danger')
save = widgets.Button(description='save', button_style='primary')
layout = widgets.HBox([slider, previous, next, inference, save])
layout_g = widgets.HBox([slider_grasps, previous_g, next_g, vis])
w = widgets.Image(width=640, height=480)
image_box = widgets.VBox([w])
tab = widgets.VBox(children = [dump])
display(image_box, layout, tab, layout_g)

slider.observe(handle_slider_change, names='value')
next.on_click(on_next_clicked)
previous.on_click(on_previous_clicked)
next_g.on_click(on_next_clicked_g)
previous_g.on_click(on_previous_clicked_g)
save.on_click(on_save_clicked)
vis.on_click(on_vis_clicked)
slider_grasps.observe(handle_slider_grasps_change)

NameError: name 'dataset_size' is not defined